# Scraping "Chronik flüchtlingsfeindlicher Vorfälle"

## Setup

Modules to be installed are numpy and pandas for handling the data, as well as requests-html for scraping the website.

In [1]:
%%capture
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests-html
!{sys.executable} -m pip install plotnine

In [2]:
import numpy as np
import pandas as pd
from requests_html import HTMLSession

## Prepare the scraper

First thing to do is to establish a html session to the website.

In [3]:
session = HTMLSession()
r = session.get('https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle')

Next, we define the urls to be scraped. In total, there are 903 sites, the first one without a suffix, later on with a suffix indicating the site number which makes it straightforward to set the full urls.

In [4]:
# Initialize lists
suffixes = [''] 
urls = []

# Set suffix of url
for i in range(1, 902):
    suffixes.append('?page=' + str(i))

# Set full urls
for suffix in suffixes:
    urls.append('https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle' + suffix)

Let's print the first five urls to check if it worked out.

In [5]:
print(*urls[0:5], sep='\n')

https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle
https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle?page=1
https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle?page=2
https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle?page=3
https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle?page=4


The urls look fine, now we ready to launch the scraper. It loops over every entry and scrapes information on each of the respective fields. If there is no information on a variable in a certain entry, the value None is assigned.

In [6]:
data = []

for n in urls:
    
    r = session.get(n)
    
    for element in r.html.find('.node-chronik-eintrag'):

        if element.find('.field-name-field-date', first = True) == None:
            date = None
        else:
            date = element.find('.field-name-field-date', first = True).text

        if element.find('.field-name-field-art', first = True) == None:
            category = None
        else:
            category = element.find('.field-name-field-art', first = True).text

        if element.find('.field-name-field-anzahl-verletze', first = True) == None:
            casualties = None 
        else:
            casualties = element.find('.field-name-field-anzahl-verletze', first = True).text               

        if element.find('.field-name-field-city', first = True) == None:
            city = None 
        else:
            city = element.find('.field-name-field-city', first = True).text

        if element.find('.field-name-field-bundesland', first = True) == None:
            bundesland = None        
        else:
            bundesland = element.find('.field-name-field-bundesland', first = True).text
        
        if element.find('a[href^="http"]', first = True) == None:
            source = None 
        else:
            source = element.find('a[href^="http"]', first = True).text
        
        if element.find('a[href^="http"]', first = True) == None:
            source_url = None 
        else:
            source_url = element.find('a[href^="http"]', first = True).links
        
        if element.find('.group-right', first = True) == None:
            description = None  
        else:
            description = element.find('.group-right', first = True).text
            
        data.append({'date': date, 
                     'category': category, 
                     'casualties': casualties, 
                     'city': city, 
                     'bundesland': bundesland,
                     'source': source, 
                     'source_url': source_url, 
                     'description': description})

Let's print the first three records to take a look at the scraping output.

In [7]:
print(*data[0:3], sep='\n')

{'date': '17.05.2019', 'category': 'Tätlicher Übergriff/Körperverletzung', 'casualties': '1Verletzte_r', 'city': 'Prenzlau', 'bundesland': 'Brandenburg', 'source': 'Nordkurier', 'source_url': {'https://www.nordkurier.de/uckermark/junge-maenner-in-prenzlau-randalieren-1935542005.html'}, 'description': 'Zwei Deutsche haben am Abend zunächst neben einer Asylunterkunft randaliert. Als Kinder, die in der Asylunterkunft leben, sie aufforderten, dies zu unterlassen, betraten die beiden 21- bzw. 23-Jährigen das Gelände der Unterkunft. Einer von ihnen zückte ein Messer und soll laut Polizei "Stichbewegungen gegen einen tschetschenischen Bewohner ausgeführt haben. Bei der folgenden Rangelei verletzte sich der Tschetschene an der Hand, ein Deutscher erlitt Verletzungen am Bein und musste operiert werden", so die Polizei weiter. Die Kriminalpolizei ermittelt.'}
{'date': '04.05.2019', 'category': 'Tätlicher Übergriff/Körperverletzung', 'casualties': None, 'city': 'Querfurt', 'bundesland': 'Sachsen-

## Turn into a dataframe and save as csv file

We use pandas methods to turn the data records into a data frame and to save it as a csv file:

In [8]:
cols = ['date', 'category', 'city', 'bundesland', 'casualties', 'description', 'source', 'source_url']
df = pd.DataFrame.from_records(data, columns=cols)
df.to_csv('data/mut_gegen_rechte_gewalt.csv', index=False)